In [596]:
# import sys
# !pip install --prefix {sys.prefix} ipywidgets

In [1]:
from ipywidgets import Text, ToggleButton, ToggleButtons, FloatSlider, HBox, Label
from ipywidgets import IntSlider, Layout, Button, Box, AppLayout, IntProgress
import time

#### Nagłówek: konfiguracja

In [75]:
confHeader = ToggleButton(
    description = 'Konfiguracja',
    disabled = False,
    button_style = 'danger', 
    layout = Layout(width = '72%', height = '50px')
)

#### Slidery: wielkość populacji, ilość bitów, liczba epok

In [76]:
populationSl = HBox([
    Label(
        'Liczba osobników populacji', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 200, step = 1, value = 20, 
        layout = Layout(width = '50%'),
    )
])

bitsSl = HBox([
    Label(
        'Dokłandność chromosomów', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 20, step = 1, value = 8, 
        layout = Layout(width = '50%'),
    )
])

epochSl = HBox([
    Label(
        'Liczba epok', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 500, step = 1, value = 100, 
        layout = Layout(width = '50%'),
    )
])

In [77]:
confSliders = Box(
    children = [populationSl, bitsSl, epochSl], 
    layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
)
configLayout = AppLayout(header = confHeader, center = confSliders)

#### Nagłówek: operatory genetyczne

In [78]:
genHeader = ToggleButton(
    description = 'Operatory genetyczne',
    disabled = False,
    button_style = 'danger', 
    layout = Layout(width = '72%', height = '50px')
)

#### Przyciski: selekcja, krzyżowanie, mutacja

In [79]:
selectionBtn = ToggleButtons(
    options = ['najlepszych', 'kołem ruletki', 'turniejowa', 'strategia elitarna'],
    description = 'Selekcja: ',
    button_style = 'danger',
)

crossoverBtn = ToggleButtons(
    options = ['jednopunktowe', 'dwupunktowe', 'trzypunktowe', 'jednorodne'],
    description = 'Krzyżowanie: ',
    button_style = 'warning',
)

mutationBtn = ToggleButtons(
    options = ['jednopunktowa', 'dwupunktowa', 'brzegowa'],
    description = 'Mutacja: ',
    button_style = 'success',
)

#### Slidery: selekcja, krzyżowanie, mutacja

In [80]:
selectionSl = HBox([
    Label(
        'Procent osobników', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 100, step = 1, value = 30, 
        layout = Layout(width = '50%'), 
    )
])

crossoverSl = HBox([
    Label(
        'Prawdopodobieństwo krzyżowania', 
        layout = Layout(width = '22%')), 
    FloatSlider(
        min = 0, max = 1, step = 0.1, value = 0.9, 
        layout = Layout(width = '50%'),
    )
])

mutationSl = HBox([
    Label(
        'Prawdopodobieństwo mutacji', 
        layout = Layout(width = '22%')), 
    FloatSlider(
        min = 0, max = 1, step = 0.1, value = 0.2, 
        layout = Layout(width = '50%'),
    )
])

#### Layout: operatory genetyczne

In [81]:
buttons = Box(
    children = [selectionBtn, crossoverBtn, mutationBtn], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)
geneticSliders = Box(
    children = [selectionSl, crossoverSl, mutationSl], 
    layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
)
box = Box(
    children = [buttons, geneticSliders], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

geneticLayout = AppLayout(header = genHeader, center = box)

In [82]:
startButton = ToggleButton(
    description = 'Rozpocznij',
    disabled = False,
    button_style = 'success', 
    layout = Layout(width = '72%', height = '50px')
)
firstPanel = Box(
    children = [configLayout, geneticLayout, startButton], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

In [83]:
epochProgress = IntProgress(
    value = 0,
    min = 0,
    max = 10,
    step = 1,
    description = 'Epoka:',
    bar_style = '',
    layout = Layout(display = 'none')
)

In [84]:
def incrementEpochProgress():
    epochProgress.value += 1

In [85]:
def updateSelectionSlider(minimum, text):
    selectionSl.layout.display = 'block'
    selectionSl.layout.display = 'flex'
    selectionSl.layout.flex_flow = 'row'
    selectionSl.children[1].min = minimum
    selectionSl.children[0].value = text

def selectionBtnClicked(change):
    if change['new'] == 'najlepszych' or change['new'] == 'strategia elitarna':
        updateSelectionSlider(0, 'Procent osobników')
    if change['new'] == 'kołem ruletki':
        selectionSl.layout.display = 'none'
    if change['new'] == 'turniejowa':
        updateSelectionSlider(1, 'Wielkość turnieju')

selectionBtn.observe(selectionBtnClicked, 'value')

In [86]:
def startApp(change):
    popSize = populationSl.children[1].value
    chromoBits = bitsSl.children[1].value
    epoch = epochSl.children[1].value
    selection = selectionBtn.value
    crossover = crossoverBtn.value
    mutation = mutationBtn.value
    if selection != 'kołem ruletki':
        selectionParam = selectionSl.children[1].value
    crossoverProb = crossoverSl.children[1].value
    mutationProb = mutationSl.children[1].value  
    
    app.children[0].layout.display = 'none'
    app.children[1].layout = Layout()
    epochProgress.max = epoch
    
    for i in range(epoch):
        time.sleep(0.1)
        incrementEpochProgress()
    

startButton.observe(startApp, 'value')

#### Rezultaty: czas obliczeń, buttony do wyświetlenia wykresów, zapis wyników do pliku

In [87]:
timeOutput = HBox([
    Label(
        'Czas obliczeń: ', 
        layout = Layout(width = '100px')), 
    Label(
        '00.00 sek', 
        layout = Layout(width = '100px'))
])

In [88]:
boothFuncBtn = ToggleButton(
    description = 'Wykres zależności wartości funkcji celu od kolejnej iteracji',
    disabled = False,
    value = False,
    button_style = 'danger',
    layout = Layout(width = '50%')
)
meanBoothFuncBtn = ToggleButton(
    description = 'Wykres zależności średniej wartości funkcji celu od kolejnej iteracji',
    disabled = False,
    value = False,
    button_style = 'danger',
    layout = Layout(width = '50%')
)
stdBtn = ToggleButton(
    description = 'Wykres zależności odchylenia standardowego od kolejnej iteracji',
    disabled = False,
    value = False,
    button_style = 'danger',
    layout = Layout(width = '50%')
)

In [89]:
plotsBox = Box(
    children = [boothFuncBtn, meanBoothFuncBtn, stdBtn], 
    layout = Layout(display = 'flex', flex_flow = 'column', margin = "10px")
)

In [90]:
fileInput = HBox([
    Label(
        'Zapis wyników do pliku:', 
        layout = Layout(width = '200px')), 
    Text(
        value = '',
        placeholder = 'lokalizacja pliku',
        disabled = False,
        layout = Layout(width = '80%')
    )
])

fileButton = ToggleButton(
    description = 'Zapisz',
    button_style = 'warning',
    disabled = False,
    layout = Layout(width = '11%')
)

In [91]:
returnBtn = ToggleButton(
    description = 'Skonfiguruj ustawienia algorytmu ponownie',
    disabled = False,
    button_style = 'success',
    layout = Layout(width = '50%')
)

In [92]:
fileBox = Box(
    children = [fileInput, fileButton], 
    layout = Layout(display = 'flex', flex_flow = 'row', margin = "10px")
)

In [93]:
outputBox = Box(
    children = [timeOutput, plotsBox, fileBox, returnBtn], 
    layout = Layout(display = 'none')
)

In [94]:
def updateTime(value):
    timeOutput.children[1].value = value

updateTime('120.50 ms')

In [95]:
def showPlot():
    plt.figure()
    plt.plot([0, 1, 2, 3, 4], [0, 3, 5, 9, 11])
    plt.xlabel('Months')
    plt.ylabel('Books Read')
    plt.show()

def showFirstPlot(value):
    if value['new'] == True:
        showPlot() # to change
    boothFuncBtn.value = False

def showSecondPlot(value):
    if value['new'] == True:
        showPlot() # to change
    meanBoothFuncBtn.value = False
    
def showThreePlot(value):
    if value['new'] == True:
        showPlot() # to change
    stdBtn.value = False

In [96]:
%matplotlib qt
import matplotlib.pyplot as plt

boothFuncBtn.observe(showFirstPlot, 'value')
meanBoothFuncBtn.observe(showSecondPlot, 'value')
stdBtn.observe(showThreePlot, 'value')

In [97]:
def restartApp(change):
    app.children[2].layout.display = 'none'
    app.children[0].layout =  Layout(display = 'flex', flex_flow = 'column')
    epochProgress.value = 0
    

returnBtn.observe(restartApp, 'value')

In [98]:
def ifFinished(change):
    if change['new'] == epochProgress.max:
        app.children[1].layout.display = 'none'
        app.children[2].layout = Layout(display = 'flex', flex_flow = 'column')

epochProgress.observe(ifFinished, 'value')

In [99]:
app = Box(
    children = [firstPanel, epochProgress, outputBox], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

In [101]:
app

Box(children=(Box(children=(AppLayout(children=(ToggleButton(value=False, button_style='danger', description='…